# Column statistics

Author: Melissa

Using parquet metadata's row group statistics, we can gather information about the global minimum and maximum values, as well as the total number of null values. 

Other statistics would be tricky to gather, and would require full scans of the data. With this approach, we need only read the `_metadata` file, and we get ALL of the row group statistics for ALL files at once.

More caveats:
* This is not checked in yet. the hipscat -> HATS transition is taking precendence right now
* I think I'd also like to consider only a subset of partitions. In this way, you could explicitly only get the ones you want, or you could perform some cone search and get the statistics for partitions that overlap that cone. It would make the method more complicated, so I'd only like to do that if it's something that sounds appealing to scientist customers.
* This notebook showcases the generic method that gathers the statistics for the `_metadata` file, but I'd like to have a convenience method on the catalog that does the path manipulation (and considers only the effective set of partitions, if the above is implemented)
* Null count might not be a very useful statistics for folks. But it's free in the `_metadata` file statistics. Could add an option to suppress that column from the result I guess?

In [1]:
from hipscat.io.parquet_metadata import aggregate_column_statistics

small_sky_metadata = "/home/delucchi/git/stats/hipscat/tests/data/small_sky_order1/_metadata"

## Simple small sky example

With everyone's favorite unit test data set, we can look at some of the initial features.

- excludes "hipscat columns" by default (`_hipscat_index`, `Norder`, `Dir`, and `Npix`)
- allows specification of specific columns to include or exclude

In [2]:
aggregate_column_statistics(small_sky_metadata)

,column_names,min_value,max_value,null_count
0,id,700.0,830.0,0.0
1,ra,280.5,350.5,0.0
2,dec,-69.5,-25.5,0.0
3,ra_error,0.0,0.0,0.0
4,dec_error,0.0,0.0,0.0


In [3]:
aggregate_column_statistics(small_sky_metadata, exclude_hipscat_columns=False)

,column_names,min_value,max_value,null_count
0,id,7.000000e+02,8.300000e+02,0.0
1,ra,2.805000e+02,3.505000e+02,0.0
2,dec,-6.950000e+01,-2.550000e+01,0.0
3,ra_error,0.000000e+00,0.000000e+00,0.0
4,dec_error,0.000000e+00,0.000000e+00,0.0
5,Norder,1.000000e+00,1.000000e+00,0.0
6,Dir,0.000000e+00,0.000000e+00,0.0
7,Npix,4.400000e+01,4.700000e+01,0.0
8,_hipscat_index,1.274969e+19,1.369672e+19,0.0


In [4]:
aggregate_column_statistics(small_sky_metadata, include_columns=["ra", "dec"])

,column_names,min_value,max_value,null_count
0,ra,280.5,350.5,0.0
1,dec,-69.5,-25.5,0.0


## Where things are less pretty

In [5]:
%%time

gaia_stats = aggregate_column_statistics('https://data.lsdb.io/unstable/gaia_dr3/gaia/_metadata')

CPU times: user 4.14 s, sys: 293 ms, total: 4.44 s
Wall time: 13.9 s


In [6]:
gaia_stats

,column_names,min_value,max_value,null_count
0,solution_id,1636148068921376768,1636148068921376768,0
1,designation,Gaia DR3 1000000057322000000,Gaia DR3 999999988604363776,0
2,source_id,4295806720,6917528997577384320,0
3,random_index,0,1811709770,0
4,ref_epoch,2016.0,2016.0,0
...,...,...,...,...
147,ag_gspphot_upper,0.0001,7.4111,1340950508
148,ebpminrp_gspphot,-0.0,4.2257,1340950508
149,ebpminrp_gspphot_lower,-0.0,4.2245,1340950508
150,ebpminrp_gspphot_upper,0.0001,4.2262,1340950508


Pandas only shows the first few and last few rows, when there's a lot of data. You can get around this with a pandas option, and I think we should have a demo notebook that showcases this functionality that calls out the pandas options.

In [7]:
import pandas as pd
pd.set_option('display.max_rows', None)

In [8]:
gaia_stats

,column_names,min_value,max_value,null_count
0,solution_id,1636148068921376768,1636148068921376768,0
1,designation,Gaia DR3 1000000057322000000,Gaia DR3 999999988604363776,0
2,source_id,4295806720,6917528997577384320,0
3,random_index,0,1811709770,0
4,ref_epoch,2016.0,2016.0,0
5,ra,3.409623912662645e-07,359.999999939548,0
6,ra_error,0.0035371692,99.997635,0
7,dec,-89.99287859590359,89.99005196682685,0
8,dec_error,0.0042951643,99.97974,0
9,parallax,-187.0293963742349,768.0665391873573,343964953


In [9]:
%%time

aggregate_column_statistics('https://data.lsdb.io/unstable/ztf/zubercal/_metadata')

CPU times: user 5.79 s, sys: 318 ms, total: 6.11 s
Wall time: 27.5 s


,column_names,min_value,max_value,null_count
0,mjd,58197.13738121402,59942.22293158946,0
1,mag,-87.34929656982422,56.31169891357422,0
2,objdec,-31.591880798339844,87.52532196044922,0
3,objra,-0.0,360.0,0
4,magerr,0,65535,0
5,objectid,70080135368679987,213033390123621252,0
6,info,-1,67113780,0
7,flag,0,52,0
8,rcidin,0,63,0
9,fieldid,199,1893,0
